In [ ]:
def hour_to_date(dt):
    return dt.date()

def round_to_hour(dt):
    dt_start_of_hour = dt.replace(minute=0, second=0, microsecond=0)
    return dt_start_of_hour


def create_visualization(data_df,user_id,week_num,data_summary_user_current):
    import numpy as np
    import pandas as pd
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go
    from itertools import repeat
    import random,pickle
#     import plotly
    
    data_df=data_df.loc[data_df['Stressor'].notnull()]
    user_df=data_df.loc[data_df.user==user_id]
    user_df["start_hour"]=user_df["starttime"].apply(round_to_hour)

    user_df["end_hour"]=user_df["endtime"].apply(round_to_hour)
    with open('/home/jupyter/sneupane/MOODS/pickled_files/colors.pickle', 'rb') as handle:
        colors = pickle.load(handle)
    random.shuffle(colors)
    if len(user_df)>0:
        colors=['#ff9868', '#ff7c51', '#ff5c37', '#ff2b16', '#ec0000', '#d00000', '#b50000', '#9b0000', '#820000', '#690000']
        user_df_group=user_df.groupby(["week","day_short"],as_index=False)["counter"].count()
        max_week=max(user_df_group["week"])
        cats=['Mon', 'Tue', 'Wed', 'Thu', 'Fri','Sat','Sun']
        day_week_count={}
        for day in cats:
            week_count={}
            user_df_group_day=user_df_group.loc[user_df_group["day_short"]==day]
            if len(user_df_group_day)>0:
                week_count=dict(zip(user_df_group_day.week, user_df_group_day.counter))
                temp_week=[0 for i in range(0,max_week)]
                for week in range(0,max_week):
                    if week in week_count:
                        temp_week[week-1]=[week_count[week]]
                    else:
                        temp_week[week-1]=[None]
                    day_week_count[day]=temp_week
            else:
                day_week_count[day]=[[None] for i in range(0,max_week)]


        fig = make_subplots(rows=1, cols=7,
                            x_title='',
                            y_title='Week #',
                            horizontal_spacing=0.06,
#                             subplot_titles=('Mon', 'Tue', 'Wed', 'Thu', 'Fri','Sat','Sun'),
                            shared_yaxes=True,)
        count=1
        for day in cats:
            fig.add_trace(
                go.Heatmap(hoverongaps = False,
                    z=day_week_count[day],
                    x=[0,1],
                    y=[i for i in range(1,max_week+1)],
                           hovertemplate=


                                        "Week #: %{y}<br>" +
                                        "# of occurences : %{z}"
                                        "<extra></extra>",
                    colorscale=colors,
                           zmin=0, zmax=4,
                           colorbar = dict(thickness=7,tick0=0,title="", dtick=1)),
                row=1, col=count
            )
        #     print
            fig.update_xaxes(title_text=day, row=1, col=count)
    
            count+=1

        fig.update_yaxes(dtick=1,title="",showgrid=True)
        fig.update_xaxes(dtick=1,showticklabels=False,showgrid=False)
        
        fig.update_layout(template="plotly_white")
#         fig.layout.xaxis.color = 'Black'
#         fig.layout.yaxis.color = 'gray'
#         fig.update_xaxes(fixedrange=True)
#         fig.update_yaxes(fixedrange=True)
#         fig.layout.xaxis.gridcolor="black"
#         fig.layout.xaxis.gridwidth=0.5
#         fig.layout.xaxis.tickfont={"family":"Times New Roman","size":15}
#         fig.layout.yaxis.gridcolor="black"
#         fig.layout.yaxis.gridwidth=0.5
        fig.update_layout(hoverlabel=dict(
                                            bgcolor="white",
                                            font_size=16,
                                            font_family="Rockwell",
                                            bordercolor="Black"
                                        )
                                    )
        fig.update_layout(
                    autosize=True,

                    margin=dict(
                       l=60,
                            r=30,
                            b=40,
                            t=40,

                    ),)
        fig.update_layout(hoverlabel=dict(
                                    bgcolor="white",
                                    font_size=16,
                                    font_family="Rockwell",
                                    bordercolor="Black"
                                )
                            )
        fig.update_layout(template="plotly_white")
        data_summary_user=data_summary_user_current["HeatMap_day_week"]
      

       
       
        
        
        return "HeatMap_day_week",fig,data_summary_user
    else:
        return "","",""